Now that we have corrected for motion and we have extracted the mean activity for each region over time, we can now do some interesting dynamic analysis. One of the aims of your thesis is to compare the results from the phase analysis to the analysis of sliding window approach. 

This papers ([paper 1](http://online.liebertpub.com/doi/abs/10.1089/brain.2011.0068) and [paper 2](http://www.sciencedirect.com/science/article/pii/S1053811916307881)) will give you a better idea of how both analysis method work and explain their differences.

We will start by looking at the phase synchrony.


# Phase Analysis

In [ ]:
# Import the required libraries
from scipy.signal import hilbert
import numpy as np
import os

## Hilbert Transform

The first step of the Phase Analyis consits in doing the Hilbert Transform on the dataset. The Hilbert Transform can be used to transform real valued data into an analitical form, which we can use to explore the phase synchronisation. Take a look at [this](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.signal.hilbert.html) to see an implementation of the hilbert transform.

**Note:** It is good practice to always perform the hilbert transform in the demeaned data and to ignore the first and last time points (see this [paper]((http://online.liebertpub.com/doi/abs/10.1089/brain.2011.0068)) for an explantion). Luckly enough, due to the way our preprocessing is done if you load the filtered data the data is already demeaned.

In [ ]:
# load the preprocessed extraced ROIs and calculate the Hilbert Transform
extracted_rois_path = '/home/jdafflon/code/msc_project/patrycja/data_out/extract_rois/rois_timeseries.txt'
extracted_rois = np.genfromtxt(extracted_rois_path, delimiter=',')

In [ ]:
# Perform Hilbert tranform and exclude the first and last 10
# time points
hiltrans = hilbert(extracted_rois)
hiltrans = hiltrans[:, 10:-10]

Reduce the effects of noise on the dataset by using a small sliding window approach. To do this define a function that performs a sliding window on your hilbert transformed data

In [ ]:
# Define the function using a sliding window of 5 time points
def sliding_window(hilbert_data, window_size):
    nregions = hilbert_data.shape[0]
    ntpoints = hilbert_data.shape[1]
    slided = np.zeros((nregions, ntpoints - window_size + 1), dtype=complex)
    window = np.ones(int(window_size)) / float(window_size)
    for roi in range(nregions):
        slided[roi, :] = np.convolve(hilbert_data[roi, :],
                                    window, 'valid')
    return slided

In [ ]:
# Call your sliding window function
window_size = 5
slided_data = sliding_window(hiltrans, window_size)

## Synchrony and Metasbility

The phenomena of synchronisation have been well established to describe the physics behind chaotic systems and non-linear dynamics. Recently, however, this particular measure has been used to assess activation and dynamic transitions over different brain regions.

The phase synchrony describes how coherently the phase changes among different brain regions.

Synchrony is defined as:
    $$ R(t)e^{i\Phi(t)} = \frac{1}{N}\sum_{n=1}^{N} e^{i\Phi_n(t)}$$

Where N = the total number of regions being analysed. The level of synchrony between phase time-courses is described by R(t), in terms of how coherently phase changes over time. $\Phi_n(t)$ represents the phase angle of a specific region.

During fully synchronous behaviour, R(t) = 1 and 0 when phase across all phase time series is fully asynchronous.

To give you a feeling of what this mean, the animation below shows how the phase angle 3 toy regions (dots) change with time. The mean phase ($\frac{1}{N}\sum^{i\Phi_n}$) is represented by the arrow in the animation. As you can see, when the regions are synchronised the phase synchrony is higher when compared when they are not synchronised. 


<img src="images/animation.gif" width="400">

The formula above allows you to compute the mean phase at 2 levels:
    * Global: Where all regions are taken into account
    * Pairwise: The parameters of interest are calculated for each pair or regions

### Synchrony

Use the formula above to calculate both pairwise as well as global synchrony. 

**Note**: remember that on the pairwise analyse you are only comparing two regions (N=2)

**Question**: How can you extract the phase from the hilbert transform?  


In [ ]:
# Calculate the pairwise as well as the global synchrony for your extracted ROIs.

### Metastability

Metastability is defined as the standard deviation of synchrony.

In [ ]:
#Using the results from Synchrony calculate the global and the pairwise metastability
